In [1]:
import rasterio
import numpy as np
import os
import resnet as rs
import sys
import importlib
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import classifier_utilities as cu
import random

Using TensorFlow backend.


In [ ]:
importlib.reload(cu)

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="3";

In [3]:
landsat_datasets, sentinel_datasets, dem_datasets, label_dataset = cu.load_data()

In [4]:
batch_size = 25
epochs = 60
num_classes = len(cu.indexed_dictionary)

# input image dimensions
tile_side = 64
img_rows, img_cols = tile_side, tile_side
img_bands = landsat_datasets[0].count + sentinel_datasets[0].count + dem_datasets[0].count - 1

input_shape = (img_bands, img_rows, img_cols)
print(input_shape)

(10, 64, 64)


In [5]:
pixels = cu.gen_balanced_pixel_locations(image_datasets=landsat_datasets, train_count=16*30000,
                                            label_dataset=label_dataset, tile_size = tile_side)

100%|██████████| 5/5 [16:39<00:00, 187.07s/it]


In [6]:
random.shuffle(pixels)

In [7]:
print(len(pixels))

360534


In [8]:
train_px, val_px, test_px = cu.train_val_test_split(pixels, 0.7, 0.7)

In [9]:
print(len(train_px))
print(len(val_px))
print(len(test_px))


252373
75712
55163


In [10]:
import keras
from keras.optimizers import SGD, Adam

In [11]:
model = rs.ResnetBuilder.build_resnet_34(input_shape,num_classes)

In [12]:
opt = Adam()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(generator=cu.tile_generator(landsat_datasets, sentinel_datasets, dem_datasets, label_dataset, tile_side, tile_side, train_px, batch_size), 
                    steps_per_epoch=len(train_px) // batch_size, epochs=epochs, verbose=1,
                    validation_data=cu.tile_generator(landsat_datasets, sentinel_datasets, dem_datasets, label_dataset, tile_side, tile_side, val_px, batch_size),
                    validation_steps=len(val_px) // batch_size)

Epoch 1/60
16
16
 1355/10094 [===>..........................] - ETA: 32:01 - loss: 2.6712 - acc: 0.3304

In [ ]:
#generator_old = nw.tile_generator(landsat_datasets, sentinel_datasets, dem_datasets, label_dataset, tile_side, tile_side, train_px, 1)
#generator_new = cu.tile_generator(landsat_datasets, sentinel_datasets, dem_datasets, label_dataset, tile_side, tile_side, train_px, 1, merge=False)

In [ ]:
#while True:
#    image_o, label_o = next(generator_old) 
#    image_n, label_n = next(generator_new)
#    if not np.array_equal(label_n,label_o) or not np.array_equal(image_n, image_o):
#        print("new:{}".format(label_n))
#        print("old:{}".format(label_o))
 
    

### Evaluate on Test Pixels

In [ ]:
cu.evaluate_cnn(model, landsat_datasets, sentinel_datasets, dem_datasets, label_dataset, tile_side, test_px, old=False)

In [ ]:
buckets = cu.pixel_balance(test_px, landsat_datasets, label_dataset)
for i in range(len(cu.indexed_dictionary)):
    print("{}:{}".format(cu.indexed_dictionary[i], buckets[i]))

# Make pixels from other tiles (some overlap but general idea on error will be clearer)

In [ ]:
pixels_1 = cu.gen_balanced_pixel_locations(image_datasets=landsat_datasets[0:1], train_count = 100000, tile_size = tile_side,
                                            label_dataset=label_dataset)
pixels_2 = cu.gen_balanced_pixel_locations(image_datasets=landsat_datasets[1:2], train_count = 100000, tile_size = tile_side,
                                            label_dataset=label_dataset)
pixels_3 = cu.gen_balanced_pixel_locations(image_datasets=landsat_datasets[2:3], train_count = 100000, tile_size = tile_side,
                                            label_dataset=label_dataset)
pixels_4 = cu.gen_balanced_pixel_locations(image_datasets=landsat_datasets[3:4], train_count = 100000, tile_size = tile_side,
                                            label_dataset=label_dataset)
pixels_5 = cu.gen_balanced_pixel_locations(image_datasets=landsat_datasets[4:], train_count = 100000, tile_size = tile_side,
                                            label_dataset=label_dataset)

# Evaluate Tile 0

In [ ]:
cu.evaluate_cnn(model, landsat_datasets[0:1], sentinel_datasets[0:1], dem_datasets[0:1], label_dataset, tile_side, pixels_1)

# Evaluate Tile 1

In [ ]:
cu.evaluate_cnn(model, landsat_datasets[1:2], sentinel_datasets[1:2], dem_datasets[1:2], label_dataset, tile_side, pixels_2)

# Evaluate Tile 2

In [ ]:
cu.evaluate_cnn(model, landsat_datasets[2:3], sentinel_datasets[2:3], dem_datasets[2:3], label_dataset, tile_side, pixels_2)

# Evaluate Tile 3

In [ ]:
cu.evaluate_cnn(model, landsat_datasets[3:4], sentinel_datasets[3:4], dem_datasets[3:4], label_dataset, tile_side, pixels_2)

# Evaluate Tile 4

In [ ]:
cu.evaluate_cnn(model, landsat_datasets[4:], sentinel_datasets[4:], dem_datasets[4:], label_dataset, tile_side, pixels_2)